## Table of Contents
- [Section 1: Introduction](#section-1-introduction)
    - [Section 1.i: Historical Context](#section-1i-hisorical-context)
    - [Section 1.ii: Scope of Problem](#section-1ii-scope-of-problem)
- [Section 2: Methodology](#section-2-methodology)
    - [Section 2.i: Processing](#section-2i-processing)
    - [Section 2.ii: Preprocessing](#section-2ii-processing)
    - [Section 2.iii: Postprocessing](#section-2iii-postprocessing)
    - [Section 2.iv: Data Verification](#section-2iv-data-verification)
    - [Section 2.v: Prepare for import and save](#section-2v-prepare-import)
    - [Section 2.vi: Overall check for missing columns](#section-2iv-overall-check)
- [Section 3: Example](#section-3-example)

## Section 1: Introduction<a name="section-1-introduction"></a>

### Section 1.i: Historical Context<a name="section-1i-hisorical-context"></a>

At the inception of the [Ontario Birth Study](http://www.ontariobirthstudy.ca) in 2013, clinical data was stored on a [Medidata Rave](https://www.medidata.com/en/products/edc/) database which was housed at the [Applied Health Research Centre (AHRC)](https://www.stmichaelshospital.com/research/ahrc/) at St. Michael’s Hospital. Medidata Rave was used as it was believed that the Ontario Birth Study data would be associated with the [Ontario Health Study](https://www.ontariohealthstudy.ca/). Since the Ontario Birth Study is no longer associated with the Ontario Health Study and there were significant limitations with the Medidata Rave database, the Ontario Birth Study decided to move the clinical database from Medidata Rave to [REDCap](https://www.project-redcap.org/) in late 2019. Both the Medidata Rave and REDCap database were developed by AHRC; however, the data transfer was done independently of ARHC. 



### Section 1.ii: Scope of Problem<a name="section-1ii-scope-of-problem"></a>

This notebook outlines the data migration process from Medidata Rave to REDCap completed in May 2020. The process outlined attempts to maximize data fidelity during data migration with minimal data cleaning - it was anticipated that the data cleaning process would be done at a later date. Therefore, for the purposes of this project, it was assumed that all Medidata Rave data received from AHRC was correct (i.e. no checking between label and coded data).

Although both the Medidata Rave and REDCap database were created by AHRC, there were differences in format and the numeric coding of variables. The Rave data was given to the Ontario Birth Study in a wide format while the REDCap data needs to be in a long format to successfully import (see [Wide and narrow data](https://en.wikipedia.org/wiki/Wide_and_narrow_data) for a description). Some columns, specifically date columns, forced the person entering the data to input a value even if one was not available. These instances had to be reinterpreted so the REDCap data reflected the intentions of the source data.

## Section 2: Methodology<a name="section-2-methodology"></a>

### Section 2.i: Processing<a name="section-2i-processing"></a>
The Rave database captures data in a wide format whereas the REDCap data is a long/narrow format. The `RedcapConv` class from `obs_clinic_migration` was created to aid in the conversion from wide to long. The intializing of `RedcapConv` takes the following parameters: `ravestub_redcap_dict`, `stub_repeat`, `master_df`, `redcap_data_dict`, and, `recode_long`.
- `ravestub_redcap_dict` maps the name of the variable in REDCap to the associated variable in Rave.
- `stub_repeat` parameter is the number of occurences in the Rave dataset. For example if there is 'parameter_1' and 'parameter_2', the stub_repeat would be set to 2.
- `master_df` is the reference data frame that is being convert - in this case it is the Rave dataframe which will be converted to the REDCap format.
- `redcap_data_dict` is the data dictionary derived from REDCap and lists the coding of variables
- `recode_long` is a boolean which will recode the Rave data set (i.e. `master_df`) to the REDCap values using based on `redcap_data_dict`

The function works by identifying the columns of interest in the Rave clinic database. For example, suppose we are interested in the Rave columns 'rave_column_a' and 'rave_column_b'. Two Rave columns are availble in the Rave data set: one with the data uncoded (i.e. represented as a string), another representing the data numerically coded. In the Rave data set, the column representing the data numerically coded ended in 'STD' (e.g. 'rave_parameter_a_STD'). 

Table 1. Example raw data from Rave

| Subject | rave_column_a | rave_column_a_STD | rave_column_b | rave_column_b_STD |
|---------|---------------|-------------------|---------------|-------------------|
| 100001  | Yes           | 1                 | Male          | 1                 |
| 100002  | No            | 0                 | Male          | 1                 |
| 100003  | Yes           | 1                 | Female        | 0                 |

The uncoded column is referenced as the key in a dictionary (i.e. `ravestub_redcap_dict`) and the coded column is ignored. The corresponding dictionary value represents the name of the REDCap column. The dictionary is used as an argument for the `RedcapConv` class during initilization. With the dictionary
```python
{
        'rave_column_a': 'redcap_column_a',
        'rave_column_b': 'redcap_column_b'
}
```
Table 1 will be converted into Table 2.

Table 2. Initial processing of raw data in table 1

| obs_id | redcap_column_a | redcap_column_b |
|---------|-----------------|-----------------|
| 100001  | Yes             | Male            |
| 100002  | No              | Male            |
| 100003  | Yes             | Female          |

If `recode_long` is set to `True`, then the initilization method will reference the column name in the REDCap data dictionary (i.e. `redcap_data_dict` parameter) and make the changes accordingly. For example, if the REDCap data dictionary listed the coding of 'redcap_column_a' as '1, No | 2, Yes' and 'redcap_column_b' as '1, Male | 2, Female | 3, Unknown', recoding Table 2 would result in Table 3.

Table 3. Example of recoding table 2

| obs_id | redcap_column_a | redcap_column_b |
|---------|-----------------|-----------------|
| 100001  | 2               | 1               |
| 100002  | 1               | 1               |
| 100003  | 2               | 2               |

This example only works with columns were there is are no repeat instances (i.e. `stub_repeat` = 0). Since the Rave data is captured in a wide format, it needs to be converted to a long format before being imported into REDCap. For example, suppose the columns of interest are 'rave_column_a_1', 'rave_column_a_2', 'rave_column_b_1', and 'rave_column_b_2' where 'rave_column_a_1' and 'rave_column_a_2' are different iterations of the same parameter (similarly for 'rave_column_b_1' and 'rave_column_b_2'). The number of instances needs to be manually entered into `RedcapConv` to properly process the data. This was done because there were instances in the Rave data set where the columns were inconsistently named.



Table 4. Example raw data from Rave with iterations

| Subject | rave_column_a_1 | rave_column_a_1_STD | rave_column_a_2 | rave_column_a_2_STD | rave_column_b_1 | rave_column_b_1_STD | rave_column_b_2 | rave_column_b_2_STD |
|-|-|-|-|-|-|-|-|-|
| 100001 | Yes | 1 | No | 0 | Yes | 1 | No | 0 |
| 100002 | No | 0 | No | 0 | Yes | 1 | Yes | 1 |
| 100003 | Yes | 1 |  |  | Yes | 1 |  |  |


When `stub_repeat` is set appropriately, Table 4 is converted to Table 5.

Table 5. Initial processing of raw data in table 4

| Subject | redcap_repeat_instance | rave_column_a | rave_column_b |
|-|-|-|-|
| 100001 | 1 | Yes | Yes |
| 100002 | 1 | No | Yes |
| 100003 | 1 | Yes | Yes |
| 100001 | 2 | No | No |
| 100002 | 2 | No | Yes |

The data in Table 5 can be recoded (i.e. `recode_long`) if desired.

### Section 2.ii: Preprocessing<a name="section-2ii-processing"></a>
The Rave columns are mapped one-to-one to REDCap columns when the `RedcapConv` is initialized. However, there are instances where the one-to-one mapping did not initially exist. Two methods, `rave_date_unknown` and `create_specify_col`, were created to preprocess the Rave data set to accomidate the one-to-one mapping. 

#### `rave_date_unknown` method
The new REDCap database has a column which indicates if the date for an event was known/available. An equivalent column may not exist in the Rave database so a new column may have had to been created.

#### `create_specify_col` method
The new REDCap database has a separate column for 'please specify'. In the Rave database 'please specify' is intermingled in a column with the associated labelled data. This function separates the 'please specify' answer into a unique column based on the coded column.

### Section 2.iii: Postprocessing<a name="section-2iii-postprocessing"></a>
There are instances when the processing is insufficient. Two additional methods, `change_str` and `remove_na` were created to aid in migration.

#### `change_str` method
There were some instances where there were spelling variations between the Rave database and the REDCap dictionary which arose as part of the database creation process. For example, if the REDCap data dictionary for column 'redcap_column_a' was '1, No | 2, Yes', the values in Table 6 would not be converted as part of the normal Processing process (Section 2.i) since the labels are different. If no changes were made, the associated values would remain as strings, not converted to integers, and there would be issues during the import process.

Table 6. Example of non-standard labels


| obs_id | redcap_column_a |
|---------|-----------------|
| 100001  | YES             |
| 100002  | NO              |
| 100003  | YES             |


Once these discrepancies were identified, the `change_str` method was used to alter the spelling of the values so they can be coded correctly. Passing the following argument to the `change_str` can convert Table 6 to integers which can be imported without issues.
```
{
    'redcap_column_a': 
        {
            'YES' : 'Yes',
            'NO': 'No'   
        }
}

```

#### `remove_na` method
In some instances, converting from the wide format in Rave to the long format in REDCap resulted in additional rows with no data. The `remove_na` method removed these rows.

### Section 2.iv: Data Verification<a name="section-2iv-data-verification"></a>
Three different processes were employed to ensure the fidelity of the data: code quality assurance, data verification, and the use of REDCap's internal data processing tools
To ensure the fidelity of the data, 3 different processes were employed.


#### Code Quality Assurance
Code used to create the output files were functionally tested using `pytest`.

#### Data Verification
Double data entry process was used for [data verification](https://en.wikipedia.org/wiki/Data_verification) as suggested by technical articles ([Strange 2006, par. 7](https://www.computerworld.com/article/2553608/avoiding-data-migration-delays.html)) and academic literature ([Blumenstein 1993](https://pubmed.ncbi.nlm.nih.gov/8235176/); [Arts, de Keizer and Scheffer 2002](https://academic.oup.com/jamia/article/9/6/600/1036696)).

The data from 40 subjects was directly entered into both the Rave and REDCap database. These 40 subjects were selected to maximize the number of columns covered in the Rave data set. The first subject that was selected had the greatest number of non-empty columns in the Rave data set. The second subject selected had the greatest number of non-empty columns after ignoring the non-empty columns the first subject had. The third subject selected had the greatest number of non-empty columns after ignoring the non-empty columns the first and second subject had. This process continued until 40 subjects were selected and is outlined in `double_data_entry_subjects.py`. 

Once the Rave data was converted into a format suitable to import into REDCap, it was then compared to the data directly entered into REDCap. If any discrepancies existed, these were rectified before the import process.

#### REDCap tools
The REDCap graphical user interface provides the ability to visualize any discrepancies with the imported csv files (Figure 1). Any data that does not conform to the REDCap configuration will be flagged and the data will not be imported until the discrepancies have been resolved or the associated record is removed.


<img src="figures/data-import-warning.png">

Figure 1. Example of REDCap data import quality check

### Section 2.v: Prepare for import and save<a name="section-2v-prepare-import"></a>
To properly import the data into REDCap, additional columns need to be added. The `prep_imp` method adds three different columns: 'redcap_event_name', 'redcap_repeat_instrument', and the name of a completion column (i.e. 'redcap_sheet_name_complete'). These values are not reviewed as part of the data verification process since they are reviewed during REDCap's 
import process. Finally, the data is saved locally and manually imported using REDCap's graphical user interface so REDCap's internal data processing tools can be used.

### Section 2.vi: Overall check for missing columns<a name="section-2iv-overall-check"></a>
The data columns that were created from the Rave data were compared to the columns in REDCap. This process was used to ensure all relevant columns were migrated and accounted - there were some columns in the REDCap project which were not in the Rave database (e.g. Diabetes in Pregnancy) which were ignored.

## Section 3: Example<a name="section-3-example"></a>

In [1]:
import obs_data_sets
import obs_clinic_migration_preprocessing
import obs_clinic_migration
import pandas as pd
import numpy as np
import re

# there is a known issue https://github.com/pandas-dev/pandas/issues/9784
# where long strings get truncated
pd.set_option('display.max_colwidth', None)
# modify how dataframe looks when printed
pd.set_option('display.max_columns', 10)

#### Preprocessing

In [2]:
obs_data_sets.rave_clinic = (
    obs_clinic_migration_preprocessing.rave_date_unknown(
        obs_data_sets.rave_clinic, 'SURG_PROCS_AFTER_BASLIN_NY_', 'Yes', 
        'PROCEDURE_DT_', 2
    )
)

#### Processing

In [3]:
ante_sur_dict = {
    'SURG_PROCS_AFTER_BASLIN_NY_': 'ante_sx_yn',
    'SURG_PROC_PERF_DURN_PREG_': 'ante_sx_class',
    'SPECIFY_PROCEDURE_': 'ante_sx_spec',
    'PROCEDURE_DT_DD_': 'ante_sx_day',
    'PROCEDURE_DT_MM_': 'ante_sx_month',
    'PROCEDURE_DT_YYYY_': 'ante_sx_year',
    'INDICATION_': 'ante_sx_indication',
    'COMMENT1_': 'ante_sx_comment',
    'PROCEDURE_DT_yn_date_': 'ante_sx_date_yn'
}
    
ante_sur = obs_clinic_migration.RedcapConv(
    ante_sur_dict, 2, master_df = obs_data_sets.rave_clinic
)

Column 'ante_sx_class', variable 'Other, specify' has an issue.


#### Postprocessing

In [4]:
ante_sur.change_str(
    {
        'ante_sx_class': 
            {'Other, specify': 'Other'},
    }
)

5    66
4    10
8     3
2     2
6     1
Name: ante_sx_class, dtype: int64


#### Data Verification
When there are no discrepancies, the dataframe will have no rows. If there are issues, the dataframe will indicate where the differences lie.

In [5]:
ante_sur.compare_conv_dde(obs_data_sets.redcap_clinic)

,obs_id,redcap_repeat_instance,ante_sx_yn,ante_sx_class,ante_sx_spec,...,ante_sx_month,ante_sx_year,ante_sx_indication,ante_sx_comment,Source


#### Prepare for import and save

In [6]:
ante_sur.prep_imp(
    'antenatal_arm_1', 
    'surgical_procedures_performed_during_pregnancy_complete', 
    'surgical_procedures_performed_during_pregnancy'
)    
ante_sur.data.to_csv('../data/processed/ante_sur.csv', index = False)